# Google Analytics user_prediction challenge 

## Cleaning Data

In [1]:
# download the libraries
import pandas as pd
from pandas import Series,DataFrame
import warnings
from copy import deepcopy
import os
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import random
%matplotlib inline

In [2]:
# load both of the datasets
traindata = pd.read_pickle('train_flat_no_hits.pkl')
testdata = pd.read_pickle('test_flat_no_hits.pkl')

In [3]:
# add a marker so that you know which dataset is which
traindata['dataset'] = 'train'
testdata['dataset'] = 'test'


In [4]:
# See train columns
traindata.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'socialEngagementType',
       'visitId', 'visitNumber', 'visitStartTime', 'device.browser',
       'device.browserVersion', 'device.browserSize', 'device.operatingSystem',
       'device.operatingSystemVersion', 'device.isMobile',
       'device.mobileDeviceBranding', 'device.mobileDeviceModel',
       'device.mobileInputSelector', 'device.mobileDeviceInfo',
       'device.mobileDeviceMarketingName', 'device.flashVersion',
       'device.language', 'device.screenColors', 'device.screenResolution',
       'device.deviceCategory', 'geoNetwork.continent',
       'geoNetwork.subContinent', 'geoNetwork.country', 'geoNetwork.region',
       'geoNetwork.metro', 'geoNetwork.city', 'geoNetwork.cityId',
       'geoNetwork.networkDomain', 'geoNetwork.latitude',
       'geoNetwork.longitude', 'geoNetwork.networkLocation', 'totals.visits',
       'totals.hits', 'totals.pageviews', 'totals.bounces', 'totals.newVisits',
       'totals.sessionQuality

In [5]:
# See test columns
testdata.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'socialEngagementType',
       'visitId', 'visitNumber', 'visitStartTime', 'device.browser',
       'device.browserVersion', 'device.browserSize', 'device.operatingSystem',
       'device.operatingSystemVersion', 'device.isMobile',
       'device.mobileDeviceBranding', 'device.mobileDeviceModel',
       'device.mobileInputSelector', 'device.mobileDeviceInfo',
       'device.mobileDeviceMarketingName', 'device.flashVersion',
       'device.language', 'device.screenColors', 'device.screenResolution',
       'device.deviceCategory', 'geoNetwork.continent',
       'geoNetwork.subContinent', 'geoNetwork.country', 'geoNetwork.region',
       'geoNetwork.metro', 'geoNetwork.city', 'geoNetwork.cityId',
       'geoNetwork.networkDomain', 'geoNetwork.latitude',
       'geoNetwork.longitude', 'geoNetwork.networkLocation', 'totals.visits',
       'totals.hits', 'totals.pageviews', 'totals.timeOnSite',
       'totals.sessionQualityDim', 'totals.new

In [6]:
# make sure the columns are in the same order in both train and test.
# the deepcopy is just to have a copy of the original even after joining them
col_order = list(set(traindata.columns.tolist() + testdata.columns.tolist()))
traindata = deepcopy(traindata[col_order])
testdata = deepcopy(testdata[col_order])

KeyError: "['trafficSource.campaignCode'] not in index"

In [7]:
#the test doesnt have trafficSource.campaignCode column, so i droped it from the train set
traindata.drop('trafficSource.campaignCode',axis=1,inplace= True)

In [8]:
#Repeat 
col_order = list(set(traindata.columns.tolist() + testdata.columns.tolist()))
traindata = deepcopy(traindata[col_order])
testdata = deepcopy(testdata[col_order])

In [11]:
#Merge both data set
all_data = traindata[col_order].append(testdata)

In [12]:
# view the Merged dataset
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2109926 entries, 0 to 401588
Data columns (total 60 columns):
 #   Column                                             Dtype 
---  ------                                             ----- 
 0   device.operatingSystem                             object
 1   date                                               int64 
 2   device.mobileDeviceModel                           object
 3   device.mobileDeviceMarketingName                   object
 4   totals.hits                                        object
 5   totals.visits                                      object
 6   device.screenResolution                            object
 7   geoNetwork.longitude                               object
 8   geoNetwork.region                                  object
 9   trafficSource.keyword                              object
 10  socialEngagementType                               object
 11  totals.pageviews                                   object
 12  t

In [9]:
#define a function to look for missing data
def values_missing(check):
    total = check.isnull().sum().sort_values(ascending = False) # getting the sum of null values and ordering
    percent = (check.isnull().sum() / check.isnull().count() * 100 ).sort_values(ascending = False) #getting the percent and order of null
    df_train = pd.concat([total, percent], axis=1, keys=['Total', 'Percent']) # Concatenating the total and percent
    print("Total and percent of missing values: ")
    print (df_train[~(df_train['Total'] == 0)]) # Returning values of nulls different of 0
    
    return

In [13]:
#define a function to investigate the columns  

def see_data(data, data_type=object, limit=5): #seting the function with data,show 5 uniques if available 
    n = data.select_dtypes(include=data_type) #selecting the desired data type
    for column in n.columns: #initializing the loop
        print("_________________________________________________________")
        print("Column Name ", column, ': \n', "Uniques: ", data[column].unique()[:limit], "\n",
              " | >>>> Total nulls: ", (round(data[column].isnull().sum() / len(data[column]) * 100,1)),
              " | >>>> Total unique values: ", all_data.nunique()[column]) #print the data and % of nulls)
        print("_________________________________________________________")

In [14]:
#Check missing values
values_missing(all_data)

Total and percent of missing values: 
                                                Total    Percent
totals.transactionRevenue                     2086818  98.904796
totals.totalTransactionRevenue                2086818  98.904796
totals.transactions                           2085062  98.821570
trafficSource.adwordsClickInfo.slot           2024047  95.929762
trafficSource.adwordsClickInfo.page           2024047  95.929762
trafficSource.adwordsClickInfo.adNetworkType  2024047  95.929762
trafficSource.adwordsClickInfo.isVideoAd      2024047  95.929762
trafficSource.adwordsClickInfo.gclId          2023891  95.922369
trafficSource.adContent                       1643600  77.898467
trafficSource.isTrueDirect                    1426999  67.632656
trafficSource.referralPath                    1142073  54.128581
trafficSource.keyword                         1093006  51.803049
totals.timeOnSite                             1057980  50.142991
totals.bounces                                105567

In [15]:
see_data(all_data)

_________________________________________________________
Column Name  device.operatingSystem : 
 Uniques:  ['Windows' 'Chrome OS' 'Android' 'Macintosh' 'iOS'] 
  | >>>> Total nulls:  0.0  | >>>> Total unique values:  26
_________________________________________________________
_________________________________________________________
Column Name  device.mobileDeviceModel : 
 Uniques:  ['not available in demo dataset'] 
  | >>>> Total nulls:  0.0  | >>>> Total unique values:  1
_________________________________________________________
_________________________________________________________
Column Name  device.mobileDeviceMarketingName : 
 Uniques:  ['not available in demo dataset'] 
  | >>>> Total nulls:  0.0  | >>>> Total unique values:  1
_________________________________________________________
_________________________________________________________
Column Name  totals.hits : 
 Uniques:  ['1' '2' '3' '4' '5'] 
  | >>>> Total nulls:  0.0  | >>>> Total unique values:  308
________

In [ ]:
# decision  process to drop some coluns with unique values(mostly 'not in the demoset') while some will left.

In [16]:
drop_col_list = ['socialEngagementType', 'device.browserVersion', 'device.browserSize', 'device.operatingSystemVersion',
                 'device.mobileDeviceBranding', 'device.mobileDeviceModel', 'device.mobileInputSelector', 
                 'device.mobileDeviceInfo', 'device.mobileDeviceMarketingName', 'device.flashVersion', 'device.language', 
                 'device.screenColors', 'device.screenResolution', 'geoNetwork.cityId', 'geoNetwork.latitude', 
                 'geoNetwork.longitude', 'geoNetwork.networkLocation', 'trafficSource.campaign', 'trafficSource.referralPath', 
                 'trafficSource.adwordsClickInfo.slot', 
                 'trafficSource.adwordsClickInfo.criteriaParameters', 'trafficSource.adContent', 'trafficSource.adwordsClickInfo.gclId', 
                 'customDimension.index']

In [17]:
all_data.drop(drop_col_list, axis=1, inplace=True)  #to drop them

In [18]:
# filling relevant columns

In [19]:
all_data['totals.pageviews'].fillna(0, inplace=True) #filling NA's with 1(since least for a visit to the site means 1 page seen)
all_data['totals.newVisits'].fillna(0, inplace=True) #filling NA's with 0(0 is suitable for Nan here)
all_data['totals.bounces'].fillna(0, inplace=True)   #filling NA's with 0(0 is suitable for Nan here)
all_data["totals.transactionRevenue"] = all_data["totals.transactionRevenue"].fillna(0.0).astype(float) #filling NA with zero
all_data["totals.totalTransactionRevenue"] = all_data["totals.totalTransactionRevenue"].fillna(0.0).astype(float) #filling NA with zero
all_data["totals.transactionRevenue"] = all_data["totals.transactionRevenue"].fillna(0.0).astype(float) #filling NA with zero
all_data["totals.transactions"] = all_data["totals.transactions"].fillna(0).astype(int) #filling NA with zero
all_data['trafficSource.isTrueDirect'].fillna(False, inplace=True) # filling boolean with False is its NAn
all_data['trafficSource.adwordsClickInfo.isVideoAd'].fillna(True, inplace=True) # filling boolean is its NAn
all_data['trafficSource.adwordsClickInfo.page'].fillna(0,inplace=True)
all_data['customDimension.value'].fillna('(not set)', inplace=True) # filling NA with '(not set)'
all_data['trafficSource.adwordsClickInfo.adNetworkType'].fillna('(not set)', inplace=True) # filling NA with '(not set)'
all_data['totals.timeOnSite'].fillna(0, inplace=True) #filling NA's with 0(0 is suitable for Nan here)
all_data['totals.sessionQualityDim'].fillna(0, inplace=True) #filling NA's with 0(0 is suitable for Nan here)
#To convert
all_data['totals.pageviews'] = all_data['totals.pageviews'].astype(int) # setting numerical column as integer
all_data['totals.newVisits'] = all_data['totals.newVisits'].astype(int) # setting numerical column as integer
all_data['totals.bounces'] = all_data['totals.bounces'].astype(int)  # setting numerical column as integer
all_data["totals.hits"] = all_data["totals.hits"].astype(float) # setting numerical to float
all_data['totals.sessionQualityDim'] = all_data['totals.sessionQualityDim'].astype(int)
all_data['totals.timeOnSite'] = all_data['totals.timeOnSite'].astype(int)
all_data['date'] = pd.to_datetime(all_data["date"], format="%Y%m%d") #convert to datetime
all_data['geoNetwork.metro'].replace({"not available in demo dataset": "(not set)"}, inplace=True)
all_data['geoNetwork.metro'].replace({"not available in demo dataset": "(not set)"}, inplace=True)

In [20]:
#view the dataset after dropping and modifying
all_data.head()

,device.operatingSystem,date,totals.hits,totals.visits,geoNetwork.region,trafficSource.keyword,totals.pageviews,totals.totalTransactionRevenue,trafficSource.isTrueDirect,trafficSource.adwordsClickInfo.isVideoAd,...,totals.newVisits,fullVisitorId,device.browser,customDimension.value,device.deviceCategory,trafficSource.medium,trafficSource.adwordsClickInfo.adNetworkType,device.isMobile,trafficSource.adwordsClickInfo.page,geoNetwork.metro
0,Windows,2017-10-16,1.0,1,not available in demo dataset,water bottle,1,0.0,False,True,...,1,3162355547410993243,Firefox,EMEA,desktop,organic,(not set),False,0,(not set)
1,Chrome OS,2017-10-16,2.0,1,California,NaN,2,0.0,False,True,...,0,8934116514970143966,Chrome,North America,desktop,referral,(not set),False,0,San Francisco-Oakland-San Jose CA
2,Android,2017-10-16,2.0,1,not available in demo dataset,NaN,2,0.0,True,True,...,1,7992466427990357681,Chrome,North America,mobile,(none),(not set),True,0,(not set)
3,Windows,2017-10-16,2.0,1,not available in demo dataset,(not provided),2,0.0,False,True,...,1,9075655783635761930,Chrome,EMEA,desktop,organic,(not set),False,0,(not set)
4,Windows,2017-10-16,2.0,1,not available in demo dataset,(not provided),2,0.0,False,True,...,1,6960673291025684308,Chrome,Central America,desktop,organic,(not set),False,0,(not set)


In [21]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2109926 entries, 0 to 401588
Data columns (total 36 columns):
 #   Column                                        Dtype         
---  ------                                        -----         
 0   device.operatingSystem                        object        
 1   date                                          datetime64[ns]
 2   totals.hits                                   float64       
 3   totals.visits                                 object        
 4   geoNetwork.region                             object        
 5   trafficSource.keyword                         object        
 6   totals.pageviews                              int32         
 7   totals.totalTransactionRevenue                float64       
 8   trafficSource.isTrueDirect                    bool          
 9   trafficSource.adwordsClickInfo.isVideoAd      bool          
 10  channelGrouping                               object        
 11  visitStartTime           

In [31]:
#to save as pickle
all_data.to_csv('/Users/chris/Desktop/chris_files.csv/fulldata.csv',index=False)